In [1]:
# 라이브러리 불러오기
import cv2
import mediapipe as mp
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

In [2]:
# 10개의 제스터를 매핑해둔 데이터
gesture = {
    0:'fist', 1:'one', 2:'two', 3:'three', 4:'four', 5:'five',
    6:'six', 7:'rock', 8:'spiderman', 9:'yeah', 10:'ok'
}

# 가위바위보를 위한 제스처 
rps_gesture = {
    0: 'rock',
    5: 'paper',
    9: 'scissors'
}

In [5]:
# 손을 찾아주는 모듈
mp_hands = mp.solutions.hands
# 특징점을 연결 및 관리해주는 모듈
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(max_num_hands =2,     # 찾는 손의 개수
               min_detection_confidence=0.5, # 정확하게 손을 찾아주는 역할     (0.5 => 신뢰도)
               min_tracking_confidence=0.5)  # 손의 위치(범위)를 찾아주는 역할 (0.5 => 신뢰도)

# knn 모델 활용하여 제스처 인식모델 학습
file = np.genfromtxt('./gesture_train.csv', delimiter= ',') #데이터

# 학습용 문제 데이터 나눠주기
X_train = file[:, :-1].astype(np.float32)
y_train = file[:, -1].astype(np.float32)

# 모델 생성
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)


KNeighborsClassifier(n_neighbors=3)

In [6]:
# 캠 초기화
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret :
        continue

    img = cv2.flip(img,1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = hands.process(img) # 손 정보 추출해서 담아주기

    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) 

    if result.multi_hand_landmarks is not None:

        for res in result.multi_hand_landmarks:
            # 각 관절의 위치를 저장할 비어있는 배열 생성하기 (21, 3)
            joint = np.zeros((21,3))
            # 관절의 위치정보를 추출하기
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z] # 관절의 위치를 numpy 배열에 저장
            
            # joint: 점의 위치값 > 통해서 관절값 구하
            # joint들로 관절값 구하기
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]
            # Normalize v 유클리디안 길이
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # 관절값으로 관절 각도 구하기
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))

            # radian > degree 각도 값으로 변경 해주는 작업이 필요
            angle = np.degrees(angle)

            # 모델에게 제스처 인식 시키기
            data = np.array([angle], dtype = np.float32)
            results = knn.predict(data)
            idx = int(results)

            # 제스쳐 인식되면 표시하기
            if idx in rps_gesture.keys():
                cv2.putText(img, text=rps_gesture[idx].upper(), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            # 이미지에 각 관절끼리의 연결선 그어주기
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

    cv2.imshow('hand', img)
    if cv2.waitKey(1) == 49:
        cap.release()
        cv2.destroyAllWindows()
        break